## Read data

1. Import the sqlite 3 library and connect to factbook.db database.
2. Write a query that returns facts table and assign the results to schema.

In [12]:
#import required libraries
import sqlite3
import pandas as pd 

In [13]:
#connect to the database
conn = sqlite3.connect("factbook.db")

In [14]:
# return facts table and assign them to schema
schema = conn.execute("pragma table_info(facts);").fetchall()
for s in schema:
    print(s)

(0, 'id', 'INTEGER', 1, None, 1)
(1, 'code', 'varchar(255)', 1, None, 0)
(2, 'name', 'varchar(255)', 1, None, 0)
(3, 'area', 'integer', 0, None, 0)
(4, 'area_land', 'integer', 0, None, 0)
(5, 'area_water', 'integer', 0, None, 0)
(6, 'population', 'integer', 0, None, 0)
(7, 'population_growth', 'float', 0, None, 0)
(8, 'birth_rate', 'float', 0, None, 0)
(9, 'death_rate', 'float', 0, None, 0)
(10, 'migration_rate', 'float', 0, None, 0)
(11, 'created_at', 'datetime', 0, None, 0)
(12, 'updated_at', 'datetime', 0, None, 0)


## Create index statement

1. Use the 'create index' statement to specify a column I want an index table for and SQLite to take care of the rest. Btw, using the 'explain query plan' gives only a high-leve query plan.
2. Practice creating an index for the 'area' column called 'area_idx', so that SQLite can use the index to search for rows in 'facts'. Although this gives speed benefits, the preparation time might take longer since each index needs to be stored in the database file. So let's use this only when I know which column to use for querying. 

In [19]:
# Query plan
query_plan_one = conn.execute("explain query plan select * from facts where population > 10000 ;").fetchall()
print(query_plan_one)
conn.execute("create index if not exists pop_idx on facts(population)")
query_plan_two = conn.execute("explain query plan select * from facts where population > 10000 ;").fetchall()
print(query_plan_two)

[(0, 0, 0, 'SCAN TABLE facts')]
[(0, 0, 0, 'SEARCH TABLE facts USING INDEX pop_idx (population>?)')]


1. Practice creating indexes for multiple columns and also try to speed up queries by creating the 'name' column. First, let's try using the 'explain query plan' statement and filter more than one column to see what SQLite returns. As this make SQLite scsn a full table, let's try with using both the 'population' and 'population_growth' columns. 

In [21]:
# Check if the table meets where constraints
query_plan_three = conn.execute("explain query plan select * from facts where population > 1000000 and population_growth < 0.05;").fetchall()
print(query_plan_three)

[(0, 0, 0, 'SEARCH TABLE facts USING INDEX pop_growth_idx (population_growth<?)')]


Finding: SQLite chose 'pop_growth_idx' instead of 'pop_idx' and this seems like SQLite picked a random index out of two since there wasn't much addition to improve accuracy.

1. To create multi-column index, let's specify which of the columns I want as the primary key. 

In [20]:
# Multi column queries
conn.execute("create index if not exists pop_idx on facts(population);").fetchall()
conn.execute("create index if not exists pop_growth_idx on facts(population_growth);").fetchall()
query_plan_four = conn.execute("explain query plan select * from facts where population > 1000000 and population_growth < 0.05;").fetchall()
print(query_plan_four)

[(0, 0, 0, 'SEARCH TABLE facts USING INDEX pop_growth_idx (population_growth<?)')]


1. Let's use the same 'create index' syntax and specify two columns in the 'on' statement, which will help SQLite use binary search to find rows that match to 'population' and 'population_growth'.

In [22]:
# Specify two columns in the ON statement
conn.execute("create index if not exists pop_pop_growth_idx on facts(population, population_growth);")
query_plan_five = conn.execute("explain query plan select * from facts where population > 1000000 and population_growth < 0.05;").fetchall()
print(query_plan_five)

[(0, 0, 0, 'SEARCH TABLE facts USING INDEX pop_pop_growth_idx (population>?)')]


Finding: SQLite returned 'pop_pop_growth_idx' becase it only had to access the 'facts' table for the column values that matched to the 'where' criteria.

1. Restrict the columns in the 'select' to narrow the return in 'population' and 'population_growth'. This will make SQLite not interact with 'facts' table. This is going to be a practice for later when I need to use a larger data set. 

In [23]:
# Covering index
conn.execute("create index if not exists pop_pop_growth_idx on facts(population, population_growth);")
query_plan_six = conn.execute("explain query plan select population, population_growth from facts where population > 1000000 and population_growth < 0.05;").fetchall()
print(query_plan_six)

[(0, 0, 0, 'SEARCH TABLE facts USING COVERING INDEX pop_pop_growth_idx (population>?)')]


1. Switch the query plan. Use 'using covering index' to make SQLite use the covering index. 

In [24]:
# Covering a single column index
conn.execute("create index if not exists pop_pop_growth_idx on facts(population, population_growth);")
query_plan_seven = conn.execute("explain query plan select population from facts where population > 1000000;").fetchall()
print(query_plan_seven)

[(0, 0, 0, 'SEARCH TABLE facts USING COVERING INDEX pop_idx (population>?)')]


Finding: SQLite used 'pop_inx' to find 'population' values instead of scanning the 'facts' table.  

Conclusion: This practice was to use 'factbook.db' to test various skills using SQLite. By giving guidelines in which data set I am looking for, SQLite showed me how each task was performed and also led me to look for other ways to make the process efficient. I hope to apply this skill set to a larger data set.